# Training environment information

In [18]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            12G        3.8G        360M         16M        8.5G          9G
Swap:            0B          0B          0B


In [19]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-710c6ddf-bb6d-56ac-fd29-7ce20e2bd0be)


In [20]:
!lscpu |grep 'Model name'

Model name:          Intel(R) Xeon(R) CPU @ 2.30GHz


In [21]:
!lscpu | grep 'Core(s) per socket:'

Core(s) per socket:  1


In [22]:
!nvidia-smi

Tue Jun 14 08:20:09 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    34W / 250W |   2267MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [23]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


Preventing disconnect from Google Colab notebook:
Right-click on the connect button and paste the code to the console of the UI:


```
function ClickConnect(){
    console.log("Clicked on connect button");
    document.querySelector("colab-connect-button").click() 
 }
 setInterval(ClickConnect,60000)
```

# Setup

In [24]:
!pip install fastai==2.5.3 -q
!pip freeze

absl-py==1.1.0
alabaster==0.7.12
albumentations==0.1.12
altair==4.2.0
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arviz==0.12.1
astor==0.8.1
astropy==4.3.1
astunparse==1.6.3
atari-py==0.2.9
atomicwrites==1.4.0
attrs==21.4.0
audioread==2.1.9
autograd==1.4
Babel==2.10.1
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==5.0.0
blis==0.4.1
bokeh==2.3.3
Bottleneck==1.3.4
branca==0.5.0
bs4==0.0.1
CacheControl==0.12.11
cached-property==1.5.2
cachetools==4.2.4
catalogue==1.0.0
certifi==2022.5.18.1
cffi==1.15.0
cftime==1.6.0
chardet==3.0.4
charset-normalizer==2.0.12
click==7.1.2
cloudpickle==1.3.0
cmake==3.22.5
cmdstanpy==0.9.5
colorcet==3.0.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.4.0
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.3
cupy-cuda111==9.4.0
cvxopt==1.2.7
cvxpy==1.0.31
cycler==0.11.0
cymem==2.0.6
Cython==0.29.30
daft==0.0.4
dask==2.12.0
datascience==0.10.6
debugpy==1.0.0
decorator==4.4.2
defusedxml==0.7.1
descartes==1.1.0
di

In [25]:
# import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from fastai.vision.all import *
import sys

In [26]:
# Load EB_ML python libraries
# The following libraries are used in this notebook and should be installed in your local machine before running this notebook.
# eb_colab_utils.py
# eb_ml_battery_lib.py
# eb_ml_utils.py

# path to load external *.py files used in this notebook
# Note: in Google Colab virtual machine you shoud copy the files in "/content" folder after BEFORE running this notebook's cell
external_python_file_path="'/.'"
sys.path.append(external_python_file_path)


from eb_ml_colab_utils import get_root_path,copy_model_to_google_drive
from eb_ml_battery_lib import load_soc_dataset,generate_image_files_from_eis
from eb_ml_utils import save_model_weights,build_data_loader,build_and_train_learner,score_model

# Training Configuration

In [27]:
#configuration dictionary
config ={}

# Root working folder (local or Google Drive)
# config['ROOT_DIR'] = get_root_path("batterie")
config['ROOT_DIR'] = get_root_path("batterie")  

# Folder with dataset in CSV format
#config['DATASETS_DIR'] = config['ROOT_DIR']+"/datasets"
config['DATASETS_DIR'] = config['ROOT_DIR']+"/datasets/EIS-vs-SOC-May2022"

# List of SoC level into dataset
#config['soc_list']=['100','090','080','070','060','050','040','030','020','010']
config['soc_list']=['100','090','080','070','060','050','040','030','020','010']


# Folder to store trained model
#config['MODELS_DIR'] = config['ROOT_DIR']+"/models"
config['MODELS_DIR'] = config['ROOT_DIR']+"/models"

Running on COLAB
Mounted at /gdrive


# Data Selection

In [28]:

# Data acquition file to load from dateset folder
# battery_list=["02_1","02_2","02_3","02_4","02_5","02_6","02_7","02_8","02_9"]
battery_list=["02_4","02_5","02_6","02_7","02_8","02_9"]

# Model Training

## Single model Train/Validation split

### Single Model - Data augmentation with realistic AGW noise amount in data acquisition simulation

In [29]:
# Enable/disable image dataset generation
# Image genaration is time and resource comsuming task. 
# You need to generate image just once for every cross validation experiment
# generate_images = False
generate_images = True

# The number of epochs is a hyperparameter that defines the number times that the 
# learning algorithm will work through the entire training dataset
# n_epochs=30
n_epochs=50

# Experiment name prefix
# This prefix will be used in name of files and foldersto store the model and the results of the experiment
# Note1: if you want to run multiple experiments you need to change the name prefix
# Note2: if you want to run in a Colab virtual machine generated images and model will be stored in your Google Drive
experiment_name_prefix="Paper_EIS_single_model_BATT_02"

#TODO: nel dataset di validazione alcune classi non sono rappresentate. Usare stratified splitter
config['Splitter'] = RandomSplitter(valid_pct=0.3, seed=41)
config['rePat'] = r'^.*_(\d+).png$'


print("battery data acquisitions for training and validation: ")
print(battery_list)
model_name=experiment_name_prefix
print("model name: "+model_name)

config['ExperimentName'] = model_name
config['IMAGES_PATH'] = config['ROOT_DIR']+"/"+config['ExperimentName']
config['TEST_IMAGES_PATH'] = config['ROOT_DIR']+"/test_"+config['ExperimentName']

#GENERATE IMAGE
if(generate_images):
    dataset,feature_col_names=load_soc_dataset(battery_list,config["soc_list"],config['DATASETS_DIR'])
    generate_image_files_from_eis(dataset,feature_col_names,config['IMAGES_PATH'],config['ExperimentName'],rescale=False,DATA_AUGMENTATION_FACTOR=10)

# TRAINING
learn= build_and_train_learner(config,n_epochs=n_epochs)
#SAVE
weights_filename=save_model_weights(learn,config["MODELS_DIR"],model_name)
filename_pth= weights_filename+".pth"
copy_model_to_google_drive(filename_pth,learn.model_dir,config["MODELS_DIR"])
# SCORE MODEL
dl=build_data_loader(config)

battery data acquisitions for training and validation: 
['02_4', '02_5', '02_6', '02_7', '02_8', '02_9']
model name: Paper_EIS_single_model_BATT_02
dataset row number: 60
start image file generation. IMAGE_PATH: /gdrive/MyDrive/batterie/Paper_EIS_single_model_BATT_02
df_real shape: (60, 14)
df_img shape: (60, 14)
soc: 100
battery: 02_4
augmentation_index: 0
/gdrive/MyDrive/batterie/Paper_EIS_single_model_BATT_02/Paper_EIS_single_model_BATT_02/EIS_BATT02_4_100.png
augmentation_index: 1
/gdrive/MyDrive/batterie/Paper_EIS_single_model_BATT_02/Paper_EIS_single_model_BATT_02/EIS_BATT1100_100.png
augmentation_index: 2
/gdrive/MyDrive/batterie/Paper_EIS_single_model_BATT_02/Paper_EIS_single_model_BATT_02/EIS_BATT1200_100.png
augmentation_index: 3
/gdrive/MyDrive/batterie/Paper_EIS_single_model_BATT_02/Paper_EIS_single_model_BATT_02/EIS_BATT1300_100.png
augmentation_index: 4
/gdrive/MyDrive/batterie/Paper_EIS_single_model_BATT_02/Paper_EIS_single_model_BATT_02/EIS_BATT1400_100.png
augmentation

Valley: 1.74e-03


epoch,train_loss,valid_loss,accuracy,time
0,3.353096,2.431792,0.172222,00:05


epoch,train_loss,valid_loss,accuracy,time
0,2.294778,2.006809,0.344444,00:03
1,2.030785,1.478997,0.455556,00:03
2,1.874972,1.135057,0.600000,00:03
3,1.751684,0.888135,0.700000,00:03
4,1.610915,0.735056,0.755556,00:03
5,1.480113,0.671173,0.788889,00:03
6,1.333991,0.542478,0.827778,00:03
7,1.210931,0.564901,0.827778,00:03
8,1.088372,0.591141,0.805556,00:03
9,0.993498,0.355116,0.877778,00:04


saved filename: Paper_EIS_single_model_BATT_02_1655195067.457903_SAVED.pth
Setting-up type transforms pipelines
Found 600 items
2 datasets of sizes 420,180
Setting up Pipeline: PILBase.create
Setting up Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}

Building one sample
  Pipeline: PILBase.create
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_single_model_BATT_02/Paper_EIS_single_model_BATT_02/EIS_BATT1656_040.png
    applying PILBase.create gives
      PILImage mode=RGB size=432x288
  Pipeline: RegexLabeller -> Categorize -- {'vocab': None, 'sort': True, 'add_na': False}
    starting from
      /gdrive/MyDrive/batterie/Paper_EIS_single_model_BATT_02/Paper_EIS_single_model_BATT_02/EIS_BATT1656_040.png
    applying RegexLabeller gives
      040
    applying Categorize -- {'vocab': None, 'sort': True, 'add_na': False} gives
      TensorCategory(3)

Final sample: (PILImage mode=RGB size=432x288, TensorCategory(3))


Found 600 items
2 da

In [30]:
score_model(weights_filename,dl)

[[10  0  0  0  0  0  0  0  0  0]
 [ 0 21  0  0  0  0  0  0  0  0]
 [ 0  0 18  1  0  0  0  0  0  0]
 [ 0  0  2 22  1  0  0  0  0  0]
 [ 0  0  0  0 14  0  0  0  0  0]
 [ 0  0  0  0  0 18  0  0  0  0]
 [ 0  0  0  0  0  0 14  2  0  0]
 [ 0  0  0  0  0  0  2 17  0  0]
 [ 0  0  0  0  0  0  0  0 17  0]
 [ 0  1  2  0  0  0  0  0  0 18]]
[[1.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         1.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.94736842 0.05263158 0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.08       0.88       0.04       0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         1.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         1.
  0.         0.         0.         0.        ]
 [0.         0.         0. 

learn.validate()


Model accuracy: 0.94
learn.get_preds()


TensorBase(0.9389)